In [3]:
# --- 1. 환경 설정 셀 (최종 수정 버전) ---
import os
import sys
import time
import random
import string
import json
import numpy as np

# 작업 디렉토리 변경 및 경로 설정
workspace_path = '/workspace'
os.chdir(workspace_path)
rfdiffusion_path = os.path.join(workspace_path, 'RFdiffusion')
if rfdiffusion_path not in sys.path:
    sys.path.append(rfdiffusion_path)

# 필요한 라이브러리 임포트
from inference.utils import parse_pdb
from colabdesign.rf.utils import fix_contigs, fix_pdb
from colabdesign.shared.protein import pdb_to_string

# --- 디버깅을 위해 간소화된 'run' 함수 ---
def run(command, steps, num_designs=1, visual="none"):
    log_filename = "/workspace/rfdiffusion_log.txt"
    print(f"RFdiffusion 프로세스를 시작합니다... 모든 출력은 '{log_filename}'에 저장됩니다.")
    print("이 작업은 시간이 걸릴 수 있으며, 완료될 때까지 기다려주세요.")
    os.system(f'{command} > {log_filename} 2>&1')
    print(f"✅ RFdiffusion 프로세스 완료. '{log_filename}' 파일을 확인해주세요.")

# --- run_diffusion 함수 (최종 수정 버전) ---
def run_diffusion(contigs, path, pdb=None, iterations=50, symmetry="none", order=1, hotspot=None,
                  chains=None, add_potential=False, num_designs=1, visual="none"):
    full_path = f"outputs/{path}"; os.makedirs(full_path, exist_ok=True)
    opts = [f"inference.output_prefix={full_path}", f"inference.num_designs={num_designs}"]
    opts.append(f"+inference.schedule_directory_path={full_path}/schedules")
    if chains == "": chains = None
    
    contigs_list = contigs.replace(","," ").replace(":"," ").split()
    is_fixed = any(c[0].isalpha() for c in contigs_list)
    mode = "fixed" if is_fixed else "free"
    
    if is_fixed:
        pdb_str = pdb_to_string(pdb)
        pdb_filename = f"{full_path}/input.pdb"
        with open(pdb_filename, "w") as handle: handle.write(pdb_str)
        opts.append(f"inference.input_pdb={pdb_filename}")
    
    opts.append(f"diffuser.T={iterations}")
    if hotspot is not None and hotspot != "":
        opts.append(f"ppi.hotspot_res=[{hotspot}]")
        
    opts.append(f"'contigmap.contigs=[{' '.join(contigs_list)}]'")
    opts += ["inference.dump_pdb=True", "inference.dump_pdb_path='/dev/shm'"]
    
    print("mode:", mode); print("output:", full_path); print("contigs:", contigs_list)
    run_script_path = "/workspace/RFdiffusion/run_inference.py"
    opts_str = " ".join(opts)
    cmd = f"python {run_script_path} {opts_str}"
    print("--- 실행될 최종 명령어 ---")
    print(cmd)
    
    run(cmd, iterations, num_designs, visual=visual)
    
    # ✅ 후처리 단계를 주석 처리하여 오류 방지
    # for n in range(num_designs):
    #     pdb_file = f"{full_path}_{n}.pdb"
    #     if os.path.exists(pdb_file):
    #         with open(pdb_file,"r") as handle: pdb_str = handle.read()
    #         with open(pdb_file,"w") as handle: handle.write(fix_pdb(pdb_str, contigs_list))

    return contigs_list, 1 # 대칭(copies)을 사용하지 않으므로 1을 반환

print("✅ Setup cell is ready.")

✅ Setup cell is ready.


In [10]:
# --- 바인더 디자인 실행 셀 (수정된 버전) ---

# 1. 파라미터 설정
name = "0829-150-600sc"
contigs = "C311-391/0 100-100"
hotspot = "C331,C360,C378,C381,C323"
iterations = 150
num_designs = 600
visual = "image"

# --- 2. 업로드한 PDB 파일 경로 직접 지정 ---
# 여기에 1단계에서 업로드한 PDB 파일의 정확한 이름을 입력하세요.
pdb_filename = "4k9e_c.pdb" 
pdb = os.path.join("/workspace", pdb_filename)

# 3. RFdiffusion 실행
# --- 이 아랫부분이 수정되었습니다 ---

# 기본 실행 이름(폴더명으로 사용될)을 설정합니다.
base_name = name
output_dir = f"outputs/{base_name}"

# 만약 동일한 이름의 폴더가 이미 존재하면, 중복을 피하기 위해 랜덤 문자열을 뒤에 붙입니다.
while os.path.exists(output_dir):
    random_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
    base_name = f"{name}_{random_suffix}"
    output_dir = f"outputs/{base_name}"

# run_diffusion 함수는 이 path를 파일 이름의 접두사로 사용합니다.
# 예: path가 "8051/8051" 이면, 결과는 outputs/8051/8051_0.pdb 로 저장됩니다.
path = os.path.join(base_name, base_name)

print(f"✅ 결과가 저장될 폴더: {output_dir}")
# --- 수정된 부분 끝 ---


flags = {
    "contigs": contigs, "pdb": pdb, "iterations": int(iterations),
    "hotspot": hotspot, "path": path, "num_designs": int(num_designs),
    "visual": visual, "symmetry": "none", "order": 1, "chains": "", "add_potential": True
}

for k, v in flags.items():
    if isinstance(v, str):
        flags[k] = v.replace("'", "").replace('"', '')

contigs, copies = run_diffusion(**flags)

# --- 마지막 출력 메시지도 수정되었습니다 ---
print(f"✅ RFdiffusion 실행 완료! 결과는 '{output_dir}' 폴더에 저장되었습니다.")

✅ 결과가 저장될 폴더: outputs/0829-150-600sc
mode: fixed
output: outputs/0829-150-600sc/0829-150-600sc
contigs: ['C311-391/0', '100-100']
--- 실행될 최종 명령어 ---
python /workspace/RFdiffusion/run_inference.py inference.output_prefix=outputs/0829-150-600sc/0829-150-600sc inference.num_designs=600 +inference.schedule_directory_path=outputs/0829-150-600sc/0829-150-600sc/schedules inference.input_pdb=outputs/0829-150-600sc/0829-150-600sc/input.pdb diffuser.T=150 ppi.hotspot_res=[C331,C360,C378,C381,C323] 'contigmap.contigs=[C311-391/0 100-100]' inference.dump_pdb=True inference.dump_pdb_path='/dev/shm'
RFdiffusion 프로세스를 시작합니다... 모든 출력은 '/workspace/rfdiffusion_log.txt'에 저장됩니다.
이 작업은 시간이 걸릴 수 있으며, 완료될 때까지 기다려주세요.
✅ RFdiffusion 프로세스 완료. '/workspace/rfdiffusion_log.txt' 파일을 확인해주세요.
✅ RFdiffusion 실행 완료! 결과는 'outputs/0829-150-600sc' 폴더에 저장되었습니다.


In [2]:
# --- ProteinMPNN 및 AlphaFold2 실행 (결과 강제 저장 최종 버전) ---
import pandas as pd
import glob
import os
import re

# --- 1. 이전 단계(RFdiffusion)에서 사용한 파라미터 다시 정의 ---
# ❗️ RFdiffusion 실행 시 설정했던 값과 '똑같이' 입력해야 합니다.
name = "0829-150-600sc"  # RFdiffusion 실행 시 사용한 'name'
num_designs = 600        # RFdiffusion 실행 시 사용한 'num_designs'
contigs = "C311-391/0 100-100".split()
copies = 1
rm_aa = "C"

# --- 2. RFdiffusion 결과 폴더 경로 자동으로 찾기 ---
try:
    # 'name'으로 시작하는 폴더를 모두 찾습니다.
    search_pattern = os.path.join('outputs', f'{name}*')
    matching_dirs = [d for d in glob.glob(search_pattern) if os.path.isdir(d)]
    if not matching_dirs:
        raise FileNotFoundError
    # 그중 가장 최근에 수정된 폴더를 선택합니다.
    output_dir = max(matching_dirs, key=os.path.getmtime)
    base_name = os.path.basename(output_dir)
    # RFdiffusion 셀의 path 로직을 반영합니다.
    path = os.path.join(base_name, base_name)
    print(f"✅ RFdiffusion 결과 폴더를 성공적으로 찾았습니다: {output_dir}")
except FileNotFoundError:
    print(f"❌ RFdiffusion 결과 폴더 ('{search_pattern}')를 찾을 수 없습니다. 'name' 변수를 확인해주세요.")
    assert False, "RFdiffusion 결과 폴더를 찾지 못했습니다."

# --- 3. ProteinMPNN 및 AlphaFold2 파라미터 설정 ---
num_seqs = 8
initial_guess = True
num_recycles = 3
use_multimer = True
mpnn_sampling_temp = 0.1

print(f"총 {num_designs}개의 백본 디자인에 대해 ProteinMPNN 및 AlphaFold2를 실행합니다.")
print("--------------------------------------------------------------------------")

# --- 4. 모든 디자인에 대해 반복 실행 ---
for i in range(num_designs):
    # RFdiffusion 셀의 중첩된 경로 방식에 맞게 입력 PDB 경로를 설정합니다.
    input_pdb_path = f"outputs/{path}_{i}.pdb"
    mpnn_output_loc = os.path.join(output_dir, f"design_{i}")
    os.makedirs(mpnn_output_loc, exist_ok=True)

    if not os.path.exists(input_pdb_path):
        print(f"⚠️ {input_pdb_path} 파일을 찾을 수 없어 건너뜁니다.")
        continue

    print(f"▶️ Processing Design {i+1}/{num_designs} ({os.path.basename(input_pdb_path)})")

    opts_list = [
        f"--pdb={input_pdb_path}", f"--loc={mpnn_output_loc}", f"--contig={':'.join(contigs)}",
        f"--copies={copies}", f"--num_seqs={num_seqs}", f"--num_recycles={num_recycles}",
        f"--rm_aa={rm_aa}", f"--mpnn_sampling_temp={mpnn_sampling_temp}",
    ]
    if initial_guess: opts_list.append("--initial_guess")
    if use_multimer: opts_list.append("--use_multimer")
    opts_str = ' '.join(opts_list)

    log_file = os.path.join(mpnn_output_loc, "log.txt")
    os.system(f"python -m colabdesign.rf.designability_test {opts_str} > {log_file} 2>&1")
    print(f"✅ Design {i+1} 처리 완료.")

print("--------------------------------------------------------------------------")
print("🔄 모든 디자인 처리 완료. 로그 파일을 분석하여 결과를 강제로 생성합니다...")

# --- 5. 로그 파일을 직접 파싱하여 결과 데이터프레임 생성 ---
all_results_data = []
for i in range(num_designs):
    log_path = os.path.join(output_dir, f"design_{i}", "log.txt")
    if not os.path.exists(log_path):
        continue
    
    with open(log_path, 'r') as f:
        for line in f:
            if line.startswith("design:"):
                try:
                    parts = line.split()
                    data = {
                        'design': i,
                        'n': int(parts[1].split(':')[1]),
                        'mpnn': float(parts[2].split(':')[1]),
                        'plddt': float(parts[3].split(':')[1]),
                        'i_ptm': float(parts[4].split(':')[1]),
                        'i_pae': float(parts[5].split(':')[1]),
                        'rmsd': float(parts[6].split(':')[1]),
                        'sequence': parts[7]
                    }
                    all_results_data.append(data)
                except (IndexError, ValueError):
                    pass

if not all_results_data:
    print("⚠️ 로그 파일에서 유효한 결과 데이터를 찾을 수 없었습니다.")
else:
    final_results_df = pd.DataFrame(all_results_data)
    manual_csv_path = os.path.join(output_dir, "all_results_MANUAL.csv")
    final_results_df.to_csv(manual_csv_path, index=False)
    print(f"✅ 모든 결과 데이터를 성공적으로 추출하여 '{manual_csv_path}'에 저장했습니다.")

    final_results_df_sorted = final_results_df.sort_values(by=['plddt', 'rmsd'], ascending=[False, True])

    print("\n📊 전체 디자인 결과 요약 (상위 10개):")
    print(final_results_df_sorted.head(10).to_string(index=False))
    print("--------------------------------------------------------------------------")

    best_model_info = final_results_df_sorted.iloc[0]
    best_design_num = int(best_model_info['design'])
    best_seq_num = int(best_model_info['n'])
    best_plddt = best_model_info['plddt']
    best_rmsd = best_model_info['rmsd']
    best_model_dir = os.path.join(output_dir, f"design_{best_design_num}")
    best_model_pdb = os.path.join(best_model_dir, f"n{best_seq_num}.pdb")

    print(f"🏆 최종 Best 모델이 선정되었습니다! 🏆")
    print(f"  - 위치: {best_model_dir}")
    print(f"  - 원본 백본: design {best_design_num} (파일: {os.path.basename(path)}_{best_design_num}.pdb)")
    print(f"  - MPNN 시퀀스: sequence {best_seq_num} (파일: n{best_seq_num}.pdb)")
    print(f"  - pLDDT: {best_plddt:.2f}")
    print(f"  - RMSD: {best_rmsd:.2f}")
    print(f"\n👉 최종 PDB 파일은 '{best_model_pdb}' 에서 확인하세요.")
    print("--------------------------------------------------------------------------")

✅ RFdiffusion 결과 폴더를 성공적으로 찾았습니다: outputs/0829-150-600sc
총 600개의 백본 디자인에 대해 ProteinMPNN 및 AlphaFold2를 실행합니다.
--------------------------------------------------------------------------
▶️ Processing Design 1/600 (0829-150-600sc_0.pdb)
✅ Design 1 처리 완료.
▶️ Processing Design 2/600 (0829-150-600sc_1.pdb)
✅ Design 2 처리 완료.
▶️ Processing Design 3/600 (0829-150-600sc_2.pdb)
✅ Design 3 처리 완료.
▶️ Processing Design 4/600 (0829-150-600sc_3.pdb)
✅ Design 4 처리 완료.
▶️ Processing Design 5/600 (0829-150-600sc_4.pdb)
✅ Design 5 처리 완료.
▶️ Processing Design 6/600 (0829-150-600sc_5.pdb)
✅ Design 6 처리 완료.
▶️ Processing Design 7/600 (0829-150-600sc_6.pdb)
✅ Design 7 처리 완료.
▶️ Processing Design 8/600 (0829-150-600sc_7.pdb)
✅ Design 8 처리 완료.
▶️ Processing Design 9/600 (0829-150-600sc_8.pdb)
✅ Design 9 처리 완료.
▶️ Processing Design 10/600 (0829-150-600sc_9.pdb)
✅ Design 10 처리 완료.
▶️ Processing Design 11/600 (0829-150-600sc_10.pdb)
✅ Design 11 처리 완료.
▶️ Processing Design 12/600 (0829-150-600sc_11.pdb)
✅ Design 12

✅ Design 113 처리 완료.
▶️ Processing Design 114/600 (0829-150-600sc_113.pdb)
✅ Design 114 처리 완료.
▶️ Processing Design 115/600 (0829-150-600sc_114.pdb)
✅ Design 115 처리 완료.
▶️ Processing Design 116/600 (0829-150-600sc_115.pdb)
✅ Design 116 처리 완료.
▶️ Processing Design 117/600 (0829-150-600sc_116.pdb)
✅ Design 117 처리 완료.
▶️ Processing Design 118/600 (0829-150-600sc_117.pdb)
✅ Design 118 처리 완료.
▶️ Processing Design 119/600 (0829-150-600sc_118.pdb)
✅ Design 119 처리 완료.
▶️ Processing Design 120/600 (0829-150-600sc_119.pdb)
✅ Design 120 처리 완료.
▶️ Processing Design 121/600 (0829-150-600sc_120.pdb)
✅ Design 121 처리 완료.
▶️ Processing Design 122/600 (0829-150-600sc_121.pdb)
✅ Design 122 처리 완료.
▶️ Processing Design 123/600 (0829-150-600sc_122.pdb)
✅ Design 123 처리 완료.
▶️ Processing Design 124/600 (0829-150-600sc_123.pdb)
✅ Design 124 처리 완료.
▶️ Processing Design 125/600 (0829-150-600sc_124.pdb)
✅ Design 125 처리 완료.
▶️ Processing Design 126/600 (0829-150-600sc_125.pdb)
✅ Design 126 처리 완료.
▶️ Processing Desi

✅ Design 239 처리 완료.
▶️ Processing Design 240/600 (0829-150-600sc_239.pdb)
✅ Design 240 처리 완료.
▶️ Processing Design 241/600 (0829-150-600sc_240.pdb)
✅ Design 241 처리 완료.
▶️ Processing Design 242/600 (0829-150-600sc_241.pdb)
✅ Design 242 처리 완료.
▶️ Processing Design 243/600 (0829-150-600sc_242.pdb)
✅ Design 243 처리 완료.
▶️ Processing Design 244/600 (0829-150-600sc_243.pdb)
✅ Design 244 처리 완료.
▶️ Processing Design 245/600 (0829-150-600sc_244.pdb)
✅ Design 245 처리 완료.
▶️ Processing Design 246/600 (0829-150-600sc_245.pdb)
✅ Design 246 처리 완료.
▶️ Processing Design 247/600 (0829-150-600sc_246.pdb)
✅ Design 247 처리 완료.
▶️ Processing Design 248/600 (0829-150-600sc_247.pdb)
✅ Design 248 처리 완료.
▶️ Processing Design 249/600 (0829-150-600sc_248.pdb)
✅ Design 249 처리 완료.
▶️ Processing Design 250/600 (0829-150-600sc_249.pdb)
✅ Design 250 처리 완료.
▶️ Processing Design 251/600 (0829-150-600sc_250.pdb)
✅ Design 251 처리 완료.
▶️ Processing Design 252/600 (0829-150-600sc_251.pdb)
✅ Design 252 처리 완료.
▶️ Processing Desi

✅ Design 350 처리 완료.
▶️ Processing Design 351/600 (0829-150-600sc_350.pdb)
✅ Design 351 처리 완료.
▶️ Processing Design 352/600 (0829-150-600sc_351.pdb)
✅ Design 352 처리 완료.
▶️ Processing Design 353/600 (0829-150-600sc_352.pdb)
✅ Design 353 처리 완료.
▶️ Processing Design 354/600 (0829-150-600sc_353.pdb)
✅ Design 354 처리 완료.
▶️ Processing Design 355/600 (0829-150-600sc_354.pdb)
✅ Design 355 처리 완료.
▶️ Processing Design 356/600 (0829-150-600sc_355.pdb)
✅ Design 356 처리 완료.
▶️ Processing Design 357/600 (0829-150-600sc_356.pdb)
✅ Design 357 처리 완료.
▶️ Processing Design 358/600 (0829-150-600sc_357.pdb)
✅ Design 358 처리 완료.
▶️ Processing Design 359/600 (0829-150-600sc_358.pdb)
✅ Design 359 처리 완료.
▶️ Processing Design 360/600 (0829-150-600sc_359.pdb)
✅ Design 360 처리 완료.
▶️ Processing Design 361/600 (0829-150-600sc_360.pdb)
✅ Design 361 처리 완료.
▶️ Processing Design 362/600 (0829-150-600sc_361.pdb)
✅ Design 362 처리 완료.
▶️ Processing Design 363/600 (0829-150-600sc_362.pdb)
✅ Design 363 처리 완료.
▶️ Processing Desi

✅ Design 461 처리 완료.
▶️ Processing Design 462/600 (0829-150-600sc_461.pdb)
✅ Design 462 처리 완료.
▶️ Processing Design 463/600 (0829-150-600sc_462.pdb)
✅ Design 463 처리 완료.
▶️ Processing Design 464/600 (0829-150-600sc_463.pdb)
✅ Design 464 처리 완료.
▶️ Processing Design 465/600 (0829-150-600sc_464.pdb)
✅ Design 465 처리 완료.
▶️ Processing Design 466/600 (0829-150-600sc_465.pdb)
✅ Design 466 처리 완료.
▶️ Processing Design 467/600 (0829-150-600sc_466.pdb)
✅ Design 467 처리 완료.
▶️ Processing Design 468/600 (0829-150-600sc_467.pdb)
✅ Design 468 처리 완료.
▶️ Processing Design 469/600 (0829-150-600sc_468.pdb)
✅ Design 469 처리 완료.
▶️ Processing Design 470/600 (0829-150-600sc_469.pdb)
✅ Design 470 처리 완료.
▶️ Processing Design 471/600 (0829-150-600sc_470.pdb)
✅ Design 471 처리 완료.
▶️ Processing Design 472/600 (0829-150-600sc_471.pdb)
✅ Design 472 처리 완료.
▶️ Processing Design 473/600 (0829-150-600sc_472.pdb)
✅ Design 473 처리 완료.
▶️ Processing Design 474/600 (0829-150-600sc_473.pdb)
✅ Design 474 처리 완료.
▶️ Processing Desi

✅ Design 572 처리 완료.
▶️ Processing Design 573/600 (0829-150-600sc_572.pdb)
✅ Design 573 처리 완료.
▶️ Processing Design 574/600 (0829-150-600sc_573.pdb)
✅ Design 574 처리 완료.
▶️ Processing Design 575/600 (0829-150-600sc_574.pdb)
✅ Design 575 처리 완료.
▶️ Processing Design 576/600 (0829-150-600sc_575.pdb)
✅ Design 576 처리 완료.
▶️ Processing Design 577/600 (0829-150-600sc_576.pdb)
✅ Design 577 처리 완료.
▶️ Processing Design 578/600 (0829-150-600sc_577.pdb)
✅ Design 578 처리 완료.
▶️ Processing Design 579/600 (0829-150-600sc_578.pdb)
✅ Design 579 처리 완료.
▶️ Processing Design 580/600 (0829-150-600sc_579.pdb)
✅ Design 580 처리 완료.
▶️ Processing Design 581/600 (0829-150-600sc_580.pdb)
✅ Design 581 처리 완료.
▶️ Processing Design 582/600 (0829-150-600sc_581.pdb)
✅ Design 582 처리 완료.
▶️ Processing Design 583/600 (0829-150-600sc_582.pdb)
✅ Design 583 처리 완료.
▶️ Processing Design 584/600 (0829-150-600sc_583.pdb)
✅ Design 584 처리 완료.
▶️ Processing Design 585/600 (0829-150-600sc_584.pdb)
✅ Design 585 처리 완료.
▶️ Processing Desi

In [ ]:
# 아래 코드는 backbone만 있어도 proteinmpnn,colabdesign을 실행시켜주는 코드

In [1]:
# --- 범용 백본(Backbone) 분석 스크립트 (최종 수정 버전) ---
import pandas as pd
import glob
import os
import re

# --- 0. 작업 경로를 올바른 위치로 변경 ---
# ✅ 이 줄이 추가되어 경로 문제를 해결합니다.
workspace_path = '/workspace'
os.chdir(workspace_path)

# --- 1. 분석 정보 입력 ---
# ❗️ 재분석하고 싶은 RFdiffusion 실행의 'name'을 정확히 입력하세요.
run_name = "0828-150-600sc"

# ❗️ 해당 백본 생성 시 사용했던 'contigs'를 정확히 입력하세요.
contigs_for_run = "C311-391/0 100-100"

# ❗️ (선택) 결과가 저장될 폴더의 접미사를 지정하세요.
output_folder_suffix = "_validation"

# --- 2. MPNN 및 AlphaFold 파라미터 설정 ---
num_seqs = 8
initial_guess = True
num_recycles = 3
use_multimer = True
rm_aa = "C"
mpnn_sampling_temp = 0.1

# --- 3. 경로 설정 및 PDB 파일 개수 자동 탐지 ---
try:
    backbone_folder = f"outputs/{run_name}"
    
    pdb_files = sorted(glob.glob(os.path.join(backbone_folder, '*.pdb')))
    if not pdb_files:
        nested_pdb_files = sorted(glob.glob(os.path.join(backbone_folder, '*', '*.pdb')))
        if nested_pdb_files:
            pdb_files = nested_pdb_files
            print("INFO: 하위 폴더에서 PDB 파일을 찾았습니다.")
        else:
             raise FileNotFoundError
    
    num_designs = len(pdb_files)
    output_dir = f"{backbone_folder}{output_folder_suffix}"
    os.makedirs(output_dir, exist_ok=True)

    print(f"✅ 분석 대상 폴더: '{backbone_folder}'")
    print(f"✅ 총 {num_designs}개의 백본 PDB 파일을 찾았습니다. 검증을 시작합니다.")
    print(f"✅ 결과는 '{output_dir}' 폴더에 저장됩니다.")

except FileNotFoundError:
    print(f"❌ '{backbone_folder}' 폴더에서 PDB 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
    assert False, "입력 PDB를 찾지 못했습니다."

# --- 4. 모든 디자인에 대해 반복 실행 ---
contigs = contigs_for_run.split()
copies = 1

print("--------------------------------------------------------------------------")
for i, input_pdb_path in enumerate(pdb_files):
    mpnn_output_loc = os.path.join(output_dir, f"design_{i}")
    os.makedirs(mpnn_output_loc, exist_ok=True)

    print(f"▶️ Processing Design {i+1}/{num_designs} ({os.path.basename(input_pdb_path)})")

    opts_list = [
        f"--pdb={input_pdb_path}", f"--loc={mpnn_output_loc}", f"--contig={':'.join(contigs)}",
        f"--copies={copies}", f"--num_seqs={num_seqs}", f"--num_recycles={num_recycles}",
        f"--rm_aa={rm_aa}", f"--mpnn_sampling_temp={mpnn_sampling_temp}",
    ]
    if initial_guess: opts_list.append("--initial_guess")
    if use_multimer: opts_list.append("--use_multimer")
    opts_str = ' '.join(opts_list)

    log_file = os.path.join(mpnn_output_loc, "log.txt")
    os.system(f"python -m colabdesign.rf.designability_test {opts_str} > {log_file} 2>&1")
    print(f"✅ Design {i+1} 처리 완료.")

print("--------------------------------------------------------------------------")
print("🔄 모든 디자인 처리 완료. 로그 파일을 분석하여 결과를 강제로 생성합니다...")

# --- 5. 로그 파일을 파싱하여 결과 데이터프레임 생성 ---
all_results_data = []
for i, input_pdb_path in enumerate(pdb_files):
    log_path = os.path.join(output_dir, f"design_{i}", "log.txt")
    if not os.path.exists(log_path):
        continue
    
    with open(log_path, 'r') as f:
        for line in f:
            if line.startswith("design:"):
                try:
                    parts = line.split()
                    data = {
                        'design_num': i,
                        'backbone_name': os.path.basename(input_pdb_path),
                        'n': int(parts[1].split(':')[1]), 'mpnn': float(parts[2].split(':')[1]),
                        'plddt': float(parts[3].split(':')[1]), 'i_ptm': float(parts[4].split(':')[1]),
                        'i_pae': float(parts[5].split(':')[1]), 'rmsd': float(parts[6].split(':')[1]),
                        'sequence': parts[7]
                    }
                    all_results_data.append(data)
                except (IndexError, ValueError):
                    pass

if not all_results_data:
    print("⚠️ 로그 파일에서 유효한 결과 데이터를 찾을 수 없었습니다.")
else:
    final_results_df = pd.DataFrame(all_results_data)
    manual_csv_path = os.path.join(output_dir, "all_results_MANUAL.csv")
    final_results_df.to_csv(manual_csv_path, index=False)
    print(f"✅ 모든 결과 데이터를 성공적으로 추출하여 '{manual_csv_path}'에 저장했습니다.")

    final_results_df_sorted = final_results_df.sort_values(by=['plddt', 'rmsd'], ascending=[False, True])

    print("\n📊 전체 디자인 결과 요약 (상위 10개):")
    print(final_results_df_sorted.head(10)[['design_num', 'backbone_name', 'n', 'plddt', 'i_pae', 'rmsd']].to_string(index=False))
    print("--------------------------------------------------------------------------")

    best_model_info = final_results_df_sorted.iloc[0]
    best_design_num = int(best_model_info['design_num'])
    best_seq_num = int(best_model_info['n'])
    best_plddt = best_model_info['plddt']
    best_rmsd = best_model_info['rmsd']
    best_backbone_name = best_model_info['backbone_name']
    best_model_dir = os.path.join(output_dir, f"design_{best_design_num}")
    best_model_pdb = os.path.join(best_model_dir, f"n{best_seq_num}.pdb")

    print(f"🏆 최종 Best 모델이 선정되었습니다! 🏆")
    print(f"  - 위치: {best_model_dir}")
    print(f"  - 원본 백본: {best_backbone_name} (design_num {best_design_num})")
    print(f"  - MPNN 시퀀스: sequence {best_seq_num} (파일: n{best_seq_num}.pdb)")
    print(f"  - pLDDT: {best_plddt:.2f}")
    print(f"  - RMSD: {best_rmsd:.2f}")
    print(f"\n👉 최종 PDB 파일은 '{best_model_pdb}' 에서 확인하세요.")
    print("--------------------------------------------------------------------------")

✅ 분석 대상 폴더: 'outputs/0828-150-600sc'
✅ 총 600개의 백본 PDB 파일을 찾았습니다. 검증을 시작합니다.
✅ 결과는 'outputs/0828-150-600sc_validation' 폴더에 저장됩니다.
--------------------------------------------------------------------------
▶️ Processing Design 1/600 (0828-150-600sc_0.pdb)
✅ Design 1 처리 완료.
▶️ Processing Design 2/600 (0828-150-600sc_1.pdb)
✅ Design 2 처리 완료.
▶️ Processing Design 3/600 (0828-150-600sc_10.pdb)
✅ Design 3 처리 완료.
▶️ Processing Design 4/600 (0828-150-600sc_100.pdb)
✅ Design 4 처리 완료.
▶️ Processing Design 5/600 (0828-150-600sc_101.pdb)
✅ Design 5 처리 완료.
▶️ Processing Design 6/600 (0828-150-600sc_102.pdb)
✅ Design 6 처리 완료.
▶️ Processing Design 7/600 (0828-150-600sc_103.pdb)
✅ Design 7 처리 완료.
▶️ Processing Design 8/600 (0828-150-600sc_104.pdb)
✅ Design 9 처리 완료.
▶️ Processing Design 10/600 (0828-150-600sc_106.pdb)
✅ Design 10 처리 완료.
▶️ Processing Design 11/600 (0828-150-600sc_107.pdb)
✅ Design 11 처리 완료.
▶️ Processing Design 12/600 (0828-150-600sc_108.pdb)
✅ Design 12 처리 완료.
▶️ Processing Design 13/6